In [1]:
import os
import numpy as np
import pandas as pd

In [51]:
Group1_scores = []
Group2_scores = []
Group3_scores = []

def emotion_definer(emotions):
    emotions = emotions.split(";")
    emotions = [x.strip(' ') for x in emotions]
    emotion_score = []
    for emotion in emotions:
        emotion = emotion.lower()
        if emotion == 'happy' or emotion == 'immersed' or emotion == 'excited' or emotion == 'hyped up' or emotion == 'hypedup' or emotion == 'delighted':
            emotion_score.append(2)
        elif emotion == 'sad' or emotion == 'angry' or ('frustrated' in emotion) or ('annoyed' in emotion) or emotion == 'bored' or emotion == 'sleepy' or emotion == 'tired' or emotion == 'nervous' or ('anxious' in emotion) or emotion == 'scared':
            emotion_score.append(0)
        elif emotion == 'tense' or emotion == 'confused' or  emotion == 'relaxed' or emotion == 'intrigued' or emotion == 'interest' or emotion == 'content' or emotion == 'calm' or emotion == 'mellow' or emotion == 'alright' or emotion == 'fine' or emotion == 'other':
            emotion_score.append(1)

    if not emotion_score:
        emotion_score.append(0)

    emotion = round(np.mean(emotion_score))

    if emotion is np.nan:
        emotion = 0

    return emotion




def load_Scores(location):
    print(location)
    folder = location[:-19]
    print(folder)
    var = os.listdir(folder)
    group = ''

    if "Group 1" in location:
        group = "Group 1"
    elif "Group 2" in location:
        group = "Group 2"
    elif "Group 3" in location:
        group = "Group 3"
        folder = folder + "/"

    for file in var:
        if group in file:
            if "xlsx" in file:
                location = folder + file
                data = pd.read_excel(location)
            elif "csv" in file:
                location = folder + file
                data = pd.read_csv(location)


    if "How would you rate this performance?" in data.columns:
        order = data["Watch ID"]
        scores = data["How would you rate this performance?"]
    elif "How did you rate the performance?" in data.columns:
        order = data["Watch ID"]
        scores  = data["How did you rate the performance?"]
    elif "How would you rate this performance?（10 is the best）" in data.columns:
        order = data["ID"]
        scores  = data["How would you rate this performance?（10 is the best）"]

    if "How would you describe your feelings through the performance?" in data.columns:
        emotions = data["How would you describe your feelings through the performance?"]
    elif "How do you describe your feeling during the programme? You may make multiple choices.\n" in data.columns:
        emotions = data["How do you describe your feeling during the programme? You may make multiple choices.\n"]

    emotions = emotions.apply(emotion_definer)

    return scores, order, emotions


def match_scores_to_data(scores, scores_order, emotions, data, data_order):
    matched_scores = []
    matched_data = []
    matched_emotions = []

    # Zip the scores and order and the data and data order
    scores_zip = zip(scores_order, scores, emotions)
    data_zip = zip(data_order, data)

    # Sort the zipped lists by order
    scores_zip = sorted(scores_zip, key=lambda x: x[0])
    data_zip = sorted(data_zip, key=lambda x: x[0])

    # Unzip the sorted lists
    sorted_scores_order, sorted_scores, sorted_emotions = zip(*scores_zip)
    sorted_data_order, sorted_data = zip(*data_zip)

    # Match the scores to the data using the order of each
    for i in range(min(len(sorted_scores_order), len(sorted_data_order))):
        if sorted_scores_order[i] == sorted_data_order[i]:
            matched_scores.append(sorted_scores[i])
            matched_data.append(sorted_data[i])
            matched_emotions.append(emotions[i])
    return matched_scores, matched_data, matched_emotions


def load_data(dir):
    data = []
    order = []
    for file in os.listdir(dir):
        if file.endswith(".csv"):
            print(os.path.join(dir, file))
            df = pd.read_csv(os.path.join(dir, file),on_bad_lines='skip')['DATA']
            if not df.empty:
                order.append(int(file[:-4]))
                data.append(df)

    shortest_dat = min([len(x) for x in data])
    for i in range(len(data)):
        data[i] = data[i].iloc[:shortest_dat]
        
        # Cut each item in data to the desired length
        sampling_rate = 24
        desired_length = 60*60*sampling_rate
        start_index = (len(data[i]) - desired_length) // 4
        end_index = start_index + desired_length
        data[i] = data[i].iloc[start_index:end_index]

    return data, order



try:
    Group1_scores, Group1_scores_order, Group1_emotions = load_Scores("/mnt/ArtHeart/Data/Group 1 Scores.xlsx")
    Group2_scores,Group2_scores_order, Group2_emotions = load_Scores("/mnt/ArtHeart/Data/Group 2 Scores.xlsx")
    Group3_scores,Group3_scores_order, Group3_emotions = load_Scores("/mnt/ArtHeart/Data/Group 3 Scores.csv")
    Group2, Group2_order = load_data("/mnt/ArtHeart/Data/Group 2")
    Group3, Group3_order = load_data("/mnt/ArtHeart/Data/Group 3")
except:
    try:
        Group1_scores, Group1_scores_order, Group1_emotions = load_Scores("W:\PhD\ArtHeart\Data\Group 1 Scores.xlsx")
        Group2_scores,Group2_scores_order, Group2_emotions = load_Scores("W:\PhD\ArtHeart\Data\Group 2 Scores.xlsx")
        Group3_scores,Group3_scores_order, Group3_emotions = load_Scores("W:\PhD\ArtHeart\Data\Group 3 Scores.csv")
        Group2_processed, Group3_processed, Group2_order, Group3_order = load_data("W:\PhD\ArtHeart\Data")

    except:
        Group1_scores, Group1_scores_order, Group1_emotions = load_Scores("Data/Group 1 Scores.xlsx")
        Group2_scores,Group2_scores_order, Group2_emotions = load_Scores("Data/Group 2 Scores.xlsx")
        Group3_scores,Group3_scores_order, Group3_emotions = load_Scores("Data/Group 3 Scores.csv")
        Group2_processed, Group3_processed, Group2_order, Group3_order = load_data("Data")



#Group1_scores, Group1_processed, Group1_emotions = match_scores_to_data(Group1_scores, Group1_scores_order, Group1_emotions, Group1_processed, Group1_order)
Group2_scores, Group2_processed, Group2_emotions = match_scores_to_data(Group2_scores, Group2_scores_order, Group2_emotions, Group2, Group2_order)
Group3_scores, Group3_processed, Group3_emotions = match_scores_to_data(Group3_scores, Group3_scores_order, Group3_emotions, Group3, Group3_order)


print(len(Group2_processed))
print(Group2_emotions)

/mnt/ArtHeart/Data/Group 1 Scores.xlsx
/mnt/ArtHeart/Data/
/mnt/ArtHeart/Data/Group 2 Scores.xlsx
/mnt/ArtHeart/Data/
/mnt/ArtHeart/Data/Group 3 Scores.csv
/mnt/ArtHeart/Data
/mnt/ArtHeart/Data/Group 2/6.csv
/mnt/ArtHeart/Data/Group 2/40.csv
/mnt/ArtHeart/Data/Group 2/7.csv
/mnt/ArtHeart/Data/Group 2/5.csv
/mnt/ArtHeart/Data/Group 2/4.csv
/mnt/ArtHeart/Data/Group 2/42.csv
/mnt/ArtHeart/Data/Group 2/46.csv
/mnt/ArtHeart/Data/Group 2/1.csv
/mnt/ArtHeart/Data/Group 2/47.csv
/mnt/ArtHeart/Data/Group 2/3.csv
/mnt/ArtHeart/Data/Group 2/44.csv
/mnt/ArtHeart/Data/Group 2/2.csv
/mnt/ArtHeart/Data/Group 2/37.csv
/mnt/ArtHeart/Data/Group 2/22.csv
/mnt/ArtHeart/Data/Group 2/20.csv
/mnt/ArtHeart/Data/Group 2/34.csv
/mnt/ArtHeart/Data/Group 2/35.csv
/mnt/ArtHeart/Data/Group 2/21.csv
/mnt/ArtHeart/Data/Group 2/25.csv
/mnt/ArtHeart/Data/Group 2/31.csv
/mnt/ArtHeart/Data/Group 2/19.csv
/mnt/ArtHeart/Data/Group 2/18.csv
/mnt/ArtHeart/Data/Group 2/24.csv
/mnt/ArtHeart/Data/Group 2/32.csv
/mnt/ArtHeart/Da

In [52]:
from sklearn.preprocessing import StandardScaler

def scale_data(data):
    scaler = StandardScaler()
    for i in range(len(data)):
        data[i]= np.array(data[i]).reshape(-1,1)
        scaled_data = scaler.fit_transform(data[i])
        data[i] = scaled_data
    return data

#Group1_processed = scale_data(Group1_processed)
Group2_processed = scale_data(Group2_processed)
Group3_processed = scale_data(Group3_processed)

In [59]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

data_all = []
data_all.extend(Group2_processed)
data_all.extend(Group3_processed)
data_all = np.array(data_all)

scores_all = []
scores_all.extend(Group2_emotions)
scores_all.extend(Group3_emotions)

#data_all = np.expand_dims(data_all, axis=-1)

# Convert scores to one-hot encoding
scores_all = tf.keras.utils.to_categorical(scores_all, num_classes=3)

X_train, X_test, y_train, y_test = train_test_split(data_all, scores_all, test_size=0.2, random_state=66)

# Define the model
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(32, input_shape=X_train[0].shape),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.summary()


# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# Train the model
model.fit(X_train, y_train, epochs=100, validation_split=0.1, batch_size=4)



Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 32)                4352      
_________________________________________________________________
dense_46 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_47 (Dense)             (None, 3)                 51        
Total params: 4,931
Trainable params: 4,931
Non-trainable params: 0
_________________________________________________________________
Train on 43 samples, validate on 5 samples


2023-06-28 14:56:46.833629: W tensorflow/core/common_runtime/bfc_allocator.cc:425] Allocator (GPU_0_bfc) ran out of memory trying to allocate 512B (rounded to 512).
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
2023-06-28 14:56:46.833662: I tensorflow/core/common_runtime/bfc_allocator.cc:881] Bin (256): 	Total Chunks: 237, Chunks in use: 237. 59.2KiB allocated for chunks. 59.2KiB in use in bin. 11.1KiB client-requested in use in bin.
2023-06-28 14:56:46.833667: I tensorflow/core/common_runtime/bfc_allocator.cc:881] Bin (512): 	Total Chunks: 30, Chunks in use: 30. 15.5KiB allocated for chunks. 15.5KiB in use in bin. 14.6KiB client-requested in use in bin.
2023-06-28 14:56:46.833670: I tensorflow/core/common_runtime/bfc_allocator.cc:881] Bin (1024): 	Total Chunks: 33, Chunks in use: 33. 46.2KiB allocated for chunks. 46.2KiB in use in bin. 44.5KiB client-requested in

ResourceExhaustedError: OOM when allocating tensor of shape [1,128] and type float
	 [[node training_34/Adam/lstm_1/kernel/v/Initializer/zeros (defined at usr/local/lib/python3.8/dist-packages/tensorflow_core/python/framework/ops.py:1748) ]]

Original stack trace for 'training_34/Adam/lstm_1/kernel/v/Initializer/zeros':
  File "usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "usr/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "usr/local/lib/python3.8/dist-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "usr/local/lib/python3.8/dist-packages/traitlets/config/application.py", line 1043, in launch_instance
    app.start()
  File "usr/local/lib/python3.8/dist-packages/ipykernel/kernelapp.py", line 725, in start
    self.io_loop.start()
  File "usr/local/lib/python3.8/dist-packages/tornado/platform/asyncio.py", line 215, in start
    self.asyncio_

In [25]:
model.evaluate(X_test, y_test)


12/12 [==============================] - 0s 107us/sample - loss: 3.9396 - acc: 0.3333


[3.9396307468414307, 0.33333334]